In [ ]:

#r "nuget:Microsoft.DotNet.Interactive.SqlServer,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive.PowerShell,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive.Http,1.0.0-beta.22175.2"

#r "nuget:FSharp.Core, *-*"
#r "nuget:Plotly.NET,*-*"
#r "nuget: Plotly.NET.Interactive, *-*"
using Plotly.NET;

In [ ]:
#!connect mssql --kernel-name splSnapshots "Server=localhost;Database=SeattlePublicLibraryOpenData;Trusted_Connection=True;TrustServerCertificate=true;"

In [ ]:
#!sql-splSnapshots --name uniqueInventory

     with uniquebibnum_cte
     AS (SELECT
        reportdate,
        COUNT(DISTINCT ei.bibnum) AS bibnumCount
         FROM   elementinventory ei
         where ei.ItemCount >0
         GROUP  BY reportdate)
SELECT ubn.reportdate
	   ,ISNULL(ubn.bibnumcount, 0) AS bibnumCount
FROM   uniquebibnum_cte ubn
ORDER  BY ubn.reportdate 

In [ ]:
#!share --from sql-splSnapshots uniqueInventory
using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;


var chart = Chart2D.Chart.Column<string, int, string, string, string>(
    Name:null,
    keysValues: new List<Tuple<string,int>>(
        uniqueInventory[0].Data.Select(kvp => new Tuple<string, int>(((DateTime)kvp["reportdate"]).ToShortDateString(), int.Parse(kvp["bibnumCount"].ToString())))
    ));

var layout = Layout.init<int>(Width:1250,DragMode:StyleParam.DragMode.Pan);

#!html
<a id="uniqueInventory"></a>

#!csharp
chart.WithLayout(layout)
.WithTitle("Unique Inventory")

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name inventoryData

WITH condensedbranchcode_cte
     AS (SELECT CASE code
                  WHEN 'lock1' THEN 'hip'
                  WHEN 'lock2' THEN 'rbe'
                  WHEN 'tcs' THEN 'cen'
                  ELSE code
                END                AS branchCode,
                Min([description]) AS [Description]
         FROM   itemlocation
         GROUP  BY CASE code
                     WHEN 'lock1' THEN 'hip'
                     WHEN 'lock2' THEN 'rbe'
                     WHEN 'tcs' THEN 'cen'
                     ELSE code
                   END),
     allreportdates_cte
     AS (SELECT DISTINCT reportdate,
                         branchcode,
                         [description]
         FROM   elementdetail
                CROSS JOIN condensedbranchcode_cte),
     uniquebibnum_cte
     AS (SELECT
        reportdate,
        cbc.branchcode,
        COUNT(DISTINCT ei.bibnum) AS bibnumCount
         FROM   elementinventory ei
                INNER JOIN condensedbranchcode_cte cbc
                        ON cbc.branchcode = ei.itemlocationcode

         GROUP  BY reportdate,
                   cbc.branchcode)
SELECT ard.reportdate
       ,ard.Description
	   ,ISNULL(ubn.bibnumcount, 0) AS bibnumCount
FROM   allreportdates_cte ard
       LEFT JOIN uniquebibnum_cte ubn
              ON ubn.branchcode = ard.branchcode
                 AND ubn.reportdate = ard.reportdate
ORDER  BY ard.reportdate 

In [ ]:
#!share --from sql-splSnapshots inventoryData

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using System;
using Range = Plotly.NET.StyleParam.Range;

List<GenericChart.GenericChart> charts = new();
List<string> altLocations = new(){
    "Interlibrary Loan", "Mobile Services", "Outreach", "Storage - Ask At Desk", "Unavailable: [do Not Use!!!]", "Wa Talking Book And Braille Library"
};
List<string> lowInventoryLocations = new(){
    "*** Temporary ***", "Outreach", "Storage - Ask At Desk", "Unavailable: [do Not Use!!!]", "Wa Talking Book And Braille Library","Interlibrary Loan"
};
int highBranchRange = inventoryData[0].Data.Where(x=>x["Description"].ToString() != "Central Library").Select(x=>int.Parse(x["bibnumCount"].ToString())).Max();
int highCentralRange = inventoryData[0].Data.Where(x=>x["Description"].ToString() == "Central Library").Select(x=>int.Parse(x["bibnumCount"].ToString())).Max();
int highAltLocationsRange = inventoryData[0].Data.Where(x=> altLocations.Contains(x["Description"].ToString())).Select(x=>int.Parse(x["bibnumCount"].ToString())).Max();
int highLowInventoryLocationsRange = inventoryData[0].Data.Where(x=> lowInventoryLocations.Contains(x["Description"].ToString())).Select(x=>int.Parse(x["bibnumCount"].ToString())).Max();
var annotation = new Annotation();
int chartIndx = 1;
foreach(var g in inventoryData[0].Data.GroupBy(x=>x["Description"]).Where(x => 
    x.Key.ToString() != "Storage - Ask At Desk" //empty BranchItemCount
    )
    .OrderBy(x => 
        {
            if(x.Key.ToString() == "Central Library"){  //force Central Library first
                return 0;
            }
            else if(lowInventoryLocations.Contains(x.Key.ToString())){
                return 3;
            }
            else if (altLocations.Contains(x.Key.ToString())){
                return 2;
            }else{
                return 1;
            }
        }).ThenBy(x=>x.Key.ToString())){

            annotation = new Annotation();
            annotation.SetValue("text", g.Key.ToString());
            annotation.SetValue("showarrow", false);
            annotation.SetValue("xref", $"x{chartIndx} domain" );
            annotation.SetValue("yref", $"y{chartIndx++} domain"); //paper
            annotation.SetValue("x", .5);
            annotation.SetValue("y", 1);
            annotation.SetValue("yshift", 30);
            
            int maxRange = 0;
            
            switch(g.Key.ToString()){
                case "Central Library":
                    maxRange = highCentralRange;
                    
                    break;
                case string key when lowInventoryLocations.Contains(key):
                    
                    maxRange = highLowInventoryLocationsRange;
                    break;    
                case string key when altLocations.Contains(key):
                    maxRange = highAltLocationsRange;
                    break;
               default:
                    maxRange = highBranchRange;
                    break;
            }
            charts.Add(
                Chart2D.Chart.Column<string, int, string, string, string>(
                    Name:g.Key.ToString()  ,
                    keysValues: new List<Tuple<string,int>>(
                        g.Select(kvp => new Tuple<string, int>(((DateTime)kvp["reportdate"]).ToShortDateString(), int.Parse(kvp["bibnumCount"].ToString())))
                    )   
                )
                .WithXAxisStyle(title: Title.init(),ShowGrid:false, ShowLine:true)
                .WithYAxisStyle(title: Title.init())
                .WithAnnotation(annotation)
                .WithYAxis(LinearAxis.init<int,int,int,int,int,int>(Range:Range.NewMinMax(0,maxRange*1.05)))
            );
}

var layout = Layout.init<int>(
    Width:1750,
     Height:5750,
     DragMode:StyleParam.DragMode.Pan
     ,Grid: LayoutGrid.init(
         SubPlots:
        new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[][]
        {
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(1), StyleParam.LinearAxisId.NewY(1)) ,  },
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(2), StyleParam.LinearAxisId.NewY(2)) ,Tuple.Create(StyleParam.LinearAxisId.NewX(3), StyleParam.LinearAxisId.NewY(3))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(4), StyleParam.LinearAxisId.NewY(4)) , Tuple.Create(StyleParam.LinearAxisId.NewX(5), StyleParam.LinearAxisId.NewY(5))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(6), StyleParam.LinearAxisId.NewY(6)) , Tuple.Create(StyleParam.LinearAxisId.NewX(7), StyleParam.LinearAxisId.NewY(7))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(8), StyleParam.LinearAxisId.NewY(8)) , Tuple.Create(StyleParam.LinearAxisId.NewX(9), StyleParam.LinearAxisId.NewY(9))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(10), StyleParam.LinearAxisId.NewY(10)) , Tuple.Create(StyleParam.LinearAxisId.NewX(11), StyleParam.LinearAxisId.NewY(11))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(12), StyleParam.LinearAxisId.NewY(12)) , Tuple.Create(StyleParam.LinearAxisId.NewX(13), StyleParam.LinearAxisId.NewY(13))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(14), StyleParam.LinearAxisId.NewY(14)) , Tuple.Create(StyleParam.LinearAxisId.NewX(15), StyleParam.LinearAxisId.NewY(15))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(16), StyleParam.LinearAxisId.NewY(16)) , Tuple.Create(StyleParam.LinearAxisId.NewX(17), StyleParam.LinearAxisId.NewY(17))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(18), StyleParam.LinearAxisId.NewY(18)) , Tuple.Create(StyleParam.LinearAxisId.NewX(19), StyleParam.LinearAxisId.NewY(19))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(20), StyleParam.LinearAxisId.NewY(20)) , Tuple.Create(StyleParam.LinearAxisId.NewX(21), StyleParam.LinearAxisId.NewY(21))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(22), StyleParam.LinearAxisId.NewY(22)) , Tuple.Create(StyleParam.LinearAxisId.NewX(23), StyleParam.LinearAxisId.NewY(23))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(24), StyleParam.LinearAxisId.NewY(24)) , Tuple.Create(StyleParam.LinearAxisId.NewX(25), StyleParam.LinearAxisId.NewY(25))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(26), StyleParam.LinearAxisId.NewY(26)) , Tuple.Create(StyleParam.LinearAxisId.NewX(27), StyleParam.LinearAxisId.NewY(27))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(28), StyleParam.LinearAxisId.NewY(28)) , Tuple.Create(StyleParam.LinearAxisId.NewX(29), StyleParam.LinearAxisId.NewY(29))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(30), StyleParam.LinearAxisId.NewY(30)) , Tuple.Create(StyleParam.LinearAxisId.NewX(31), StyleParam.LinearAxisId.NewY(31))},
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(32), StyleParam.LinearAxisId.NewY(32)) , Tuple.Create(StyleParam.LinearAxisId.NewX(33), StyleParam.LinearAxisId.NewY(33))}
           }
        )
     ,Colorway:Color.fromColors(
         Enumerable.Repeat(Color.fromKeyword(ColorKeyword.DeepSkyBlue),26)
         .Prepend(Color.fromKeyword(ColorKeyword.DarkBlue))
         .Concat(Enumerable.Repeat(Color.fromKeyword(ColorKeyword.Blueviolet),1))
         .Concat(Enumerable.Repeat(Color.fromKeyword(ColorKeyword.SteelBlue),5))
         )
     ,ShowLegend:false
    );
     
    
var g = Chart.Grid<IEnumerable<GenericChart.GenericChart>>(17,2
    ,YGap:.37    
    ,XGap:.1
    )
.Invoke(charts)
.WithLayout(layout
    );

var ax = GenericChart.getLayout(g).TryGetTypedValue<DynamicObj.DynamicObj>("xaxis");
ax.Value.SetValue("anchor","y");
ax.Value.SetValue("domain", new double[]{0,1});  //first chart 100% width
#!html
<a id="panel"></a>

#!csharp
g

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name tempItems

select 
distinct bibnum, title
 from ElementDetail ed
inner join (
select * from 
Title) t on t.TitleId = ed.TitleId

where ed.BibNum in(
select 
distinct 
bibnum
from elementinventory
where ItemLocationCode = 'drp1'
)


In [ ]:
#!share --from sql-splSnapshots tempItems

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using System;
using Range = Plotly.NET.StyleParam.Range;
using ChartDomain = Plotly.NET.ChartDomain.Chart;

var header =  new string[]{"BibNum", "Title"};
var temporaryItemstable = ChartDomain.Table<string, char, IEnumerable<string>,string>(
    header
    ,tempItems[0].Data.Select(x => new string[]{x["bibnum"].ToString(), x["title"].ToString()})
    ,MultiColumnWidth: new double[]{.15,.85}
    ,CellsAlign: StyleParam.HorizontalAlign.Left
).WithLayout(
    Layout.init<int>(
        Width:750,
        Height:235,
        Margin:Margin.init<int,int,int,int,int,int>(Top:10, Left: 10, Right: 10, Bottom:10)
));


#!html
<a id="ItemsInTempLocation"></a>

#!csharp
temporaryItemstable


<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name goneCollections


;with cte as (
    select 
    ItemCollectionCode,
    reportdate,
    --count(ItemCollectioncode),
    count(itemcollectioncode) over( partition by itemcollectioncode) codeCount
    from 
    ElementInventory ei
    where ItemLocationCode = 'cen'
    and ReportDate in ('2017-08-01', '2017-09-01')
    group by ReportDate, ItemCollectionCode
    )
    select 
    
    ic.*
    from cte
    inner join ItemCollection ic on cte.ItemCollectionCode = ic.Code
    where codeCount = 1
    and ReportDate = '2017-08-01'
    order by ItemCollectionCode, ReportDate
    

In [ ]:
#!share --from sql-splSnapshots goneCollections

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using System;
using Range = Plotly.NET.StyleParam.Range;
using ChartDomain = Plotly.NET.ChartDomain.Chart;

var header =  new string[]{"Code", "Description", "FormatGroup", "FormatSubgroup", "CategoryGroup","CategorySubgroup", "AgeGroup"};
var table = ChartDomain.Table<string, char, IEnumerable<string>,string>(
    header
    ,goneCollections[0].Data.Select(x => new string[]{x[header[0]].ToString(),
     x[header[1]].ToString(), x[header[2]].ToString(), x[header[3]].ToString()
     , x[header[4]].ToString(), x[header[5]].ToString(), x[header[6]].ToString()})
    ,MultiColumnWidth: new double[]{.09,.15,.1,.13,.11,.13,.09}
    ,CellsAlign: StyleParam.HorizontalAlign.Left
).WithLayout(
    Layout.init<int>(
        Width:750,
        Height:255,
        Margin:Margin.init<int,int,int,int,int,int>(Top:10, Left: 10, Right: 20, Bottom:10)
));

#!html
<a id="GoneCollections"></a>

#!csharp
table

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div